In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pdfplumber
out_dir = '../local_extras/'
calendar_pdf_fname = os.path.join(out_dir,'UGCalendar24-25.pdf')
pdf_path = calendar_pdf_fname
import re
# DL from: https://www.concordia.ca/content/dam/concordia/offices/registrar/docs/calendar/24-25/UGCalendar24-25.pdf



In [111]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the page to scrape
url = "https://www.concordia.ca/academics/undergraduate/calendar/current/section-31-faculty-of-arts-and-science/section-31-250-department-of-psychology/psychology-courses.html"

# Send a request to fetch the HTML content
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Initialize list to hold course data
courses = []

# Find all course blocks - this depends on the page structure
# You may need to adjust these tags/classes based on the HTML layout of the page
course_blocks = soup.find_all('div', class_='courseblock')  # Adjust selector based on the HTML structure

# Loop through each course block to extract information
for block in course_blocks:
    course_code = block.find('span', class_='courseblockcode').get_text(strip=True) if block.find('span', class_='courseblockcode') else None
    title = block.find('span', class_='courseblocktitle').get_text(strip=True) if block.find('span', class_='courseblocktitle') else None
    credits = block.find('span', class_='courseblockcredits').get_text(strip=True) if block.find('span', class_='courseblockcredits') else None
    description = block.find('p', class_='courseblockdesc').get_text(strip=True) if block.find('p', class_='courseblockdesc') else None
    
    # Optional: Find additional sections like prerequisites, components, notes, if they exist
    prerequisites = block.find('p', class_='courseblockprereq').get_text(strip=True) if block.find('p', class_='courseblockprereq') else ""
    components = block.find('p', class_='courseblockcomponents').get_text(strip=True) if block.find('p', class_='courseblockcomponents') else ""
    notes = block.find('p', class_='courseblocknotes').get_text(strip=True) if block.find('p', class_='courseblocknotes') else ""
    
    # Append extracted data as a dictionary to the list
    courses.append({
        "course_code": course_code,
        "title": title,
        "credits": credits,
        "description": description,
        "prerequisites": prerequisites,
        "components": components,
        "notes": notes
    })

# Convert the list of dictionaries to a DataFrame for easier manipulation and display
courses_df = pd.DataFrame(courses)
print(courses_df.head())

# Optional: Save to a CSV file
courses_df.to_csv('concordia_psychology_courses.csv', index=False)


Empty DataFrame
Columns: []
Index: []


Empty DataFrame
Columns: []
Index: []


In [ ]:
# Save to CSV or JSON for persistence
courses_df.to_csv(os.path.join(out_dir,'courses.csv'), index=False)
courses_df.to_json(os.path.join(out_dir,'courses.json'), orient='records', indent=4)


# PDF scraping alternative (may be more robust...)

In [113]:
#pdfplumber seems to be better able to handle the raw text in the file
#however, we will need to check the output, maybe against the web?
# also takes 4x longer :( (8s)
# Regex pattern to match course details

# course_pattern = re.compile(
#     r'([A-Z]{4} \d{3})\s+(.+?)\s*\((\d+) credits\)',             # Course code, title, and credits
#     re.DOTALL
# )
# prerequisite_pattern = re.compile(r'Prerequisite/Corequisite:\s*(.+?)(?=\n[A-Z]{4} \d{3}|\Z)', re.DOTALL)
# description_pattern = re.compile(r'Description:\s*(.+?)(?=\n[A-Z]{4} \d{3}|\Z)', re.DOTALL)
# component_pattern = re.compile(r'Component\(s\):\s*(.+?)(?=\n[A-Z]{4} \d{3}|\Z)', re.DOTALL)
# notes_pattern = re.compile(r'Notes:\s*(.+?)(?=\n[A-Z]{4} \d{3}|\Z)', re.DOTALL)

# Combined regex pattern to capture all course details
course_pattern = re.compile(
    r'([A-Z]{4} \d{3})\s+(.+?)\s*\((\d+) credits\)'             # Course code, title, and credits
    r'(?:\s+Prerequisite/Corequisite:\s*(.*?)(?=\n[A-Z]{4} \d{3}|\Z))?'  # Optional prerequisites
    r'(?:\s+Description:\s*(.*?)(?=\n[A-Z]{4} \d{3}|\Z))?'              # Optional description
    r'(?:\s+Component\(s\):\s*(.*?)(?=\n[A-Z]{4} \d{3}|\Z))?'           # Optional components
    r'(?:\s+Notes:\s*(.*?)(?=\n[A-Z]{4} \d{3}|\Z))?'                    # Optional notes
    , re.DOTALL
)

# List to store extracted course information
courses = []

# Define the page range for the courses
program_class_range = range(1227, 1264)  # inclusive, now 0-based indexing

# Open and process the PDF
with pdfplumber.open(pdf_path) as pdf:
    text = ""
    for page_num in program_class_range:
        page = pdf.pages[page_num]
        text += page.extract_text()

In [123]:
for match in course_pattern.finditer(text):
    for idx in range(100):
        print(match.group(idx+1))
        
    break

NEUR 355
Fundamentals of Neuroscience
3
One of the following courses must be completed previously: BIOL 200 or BIOL 201 or BIOL 202; or Cegep equivalent.
Description:
This course provides an introduction to the neural mechanisms that underlie behaviour. Topics include the structure
and function of neurons, neural communication, an introduction to neuroanatomy and endocrinology, and the
processing of sensory information. Students also learn how complex systems, such as the sensory and motor
systems, interact to produce behaviour.
Component(s):
Lecture
Notes:
• Students who have received credit for BIOL 383 or PSYC 355 or PSYC 358 may not take this course for credit.
None
None
None


IndexError: no such group

In [87]:

# Apply the combined regex to capture each course section
for match in course_pattern.finditer(text):
    course_code = match.group(1)
    title = match.group(2).strip()
    credits = int(match.group(3))
    prerequisites = match.group(4).strip() if match.group(4) else ""
    description = match.group(5).strip() if match.group(5) else ""
    components = match.group(6).strip() if match.group(6) else ""
    notes = match.group(7).strip() if match.group(7) else ""

    # Append each course's details as a dictionary
    courses.append({
        "course_code": course_code,
        "title": title,
        "credits": credits,
        "prerequisites": prerequisites,
        "description": description,
        "components": components,
        "notes": notes
    })

# Convert the list of courses to a DataFrame
courses_df = pd.DataFrame(courses)

# Display or save the data
courses_df.to_csv('neuroscience_courses.csv', index=False)
courses_df.head()


,course_code,title,credits,prerequisites,description,components,notes
0,NEUR 355,Fundamentals of Neuroscience,3,One of the following courses must be completed...,,,
1,NEUR 356,Research Methods in Neuroanatomy,3,The following courses must be completed previo...,,,
2,NEUR 416,Large Data Analytics,3,The following courses must be completed previo...,,,
3,NEUR 451,Learning and Neuroplasticity,3,The following courses must be completed previo...,,,
4,NEUR 458,Neuroscience Advanced Issues,3,The following courses must be completed previo...,,,


In [ ]:
import fitz  # PyMuPDF
import re
import pandas as pd

# Path to your PDF file
pdf_path = calendar_pdf_fname

# Combined regex pattern to capture all course details
course_pattern = re.compile(
    r'([A-Z]{4} \d{3})\s+(.+?)\s*\((\d+) credits\)'             # Course code, title, and credits
    r'(?:\s+Prerequisite/Corequisite:\s*(.*?)(?=\n[A-Z]{4} \d{3}|\Z))?'  # Optional prerequisites
    r'(?:\s+Description:\s*(.*?)(?=\n[A-Z]{4} \d{3}|\Z))?'              # Optional description
    r'(?:\s+Component\(s\):\s*(.*?)(?=\n[A-Z]{4} \d{3}|\Z))?'           # Optional components
    r'(?:\s+Notes:\s*(.*?)(?=\n[A-Z]{4} \d{3}|\Z))?'                    # Optional notes
    , re.DOTALL
)

# List to store extracted course information
courses = []

# Specify the range of pages to process
program_class_range = range(1227, 1264)  # inclusive, now 0-based indexing
with fitz.open(pdf_path) as pdf:
    text = ""
    for page_num in program_class_range:
        page = pdf[page_num]
        text = page.get_text("text")
        break  # Get all text from the specified page range


In [83]:
text

'Psychology Courses\nNeuroscience Courses\nNEUR 355 Fundamentals of Neuroscience (3 credits)\nPrerequisite/Corequisite:\nOne of the following courses must be completed previously: BIOL 200 or BIOL 201 or BIOL 202; or Cegep equivalent.\nDescription:\nThis course provides an introduction to the neural mechanisms that underlie behaviour. Topics include the structure\nand function of neurons, neural communication, an introduction to neuroanatomy and endocrinology, and the\nprocessing of sensory information. Students also learn how complex systems, such as the sensory and motor\nsystems, interact to produce behaviour.\nComponent(s):\nLecture\nNotes:\n• Students who have received credit for BIOL 383 or PSYC 355 or PSYC 358 may not take this course for credit.\nNEUR 356 Research Methods in Neuroanatomy (3 credits)\nPrerequisite/Corequisite:\nThe following courses must be completed previously: NEUR 355; PSYC 315. Permission of the Department is required\nfor non-Neuroscience students.\nDescrip

In [ ]:
import pdfplumber
import re
import pandas as pd

# Path to your PDF file

# Regex pattern to match course details
course_pattern = re.compile(
    r'([A-Z]{4} \d{3})\s+(.+?)\s*\((\d+) credits\)',             # Course code, title, and credits
    re.DOTALL
)
prerequisite_pattern = re.compile(r'Prerequisite/Corequisite:\s*(.+?)(?=\n[A-Z]{4} \d{3}|\Z)', re.DOTALL)
description_pattern = re.compile(r'Description:\s*(.+?)(?=\n[A-Z]{4} \d{3}|\Z)', re.DOTALL)
component_pattern = re.compile(r'Component\(s\):\s*(.+?)(?=\n[A-Z]{4} \d{3}|\Z)', re.DOTALL)
notes_pattern = re.compile(r'Notes:\s*(.+?)(?=\n[A-Z]{4} \d{3}|\Z)', re.DOTALL)

# List to store extracted course information
courses = []

# Define the page range for the courses
program_class_range = range(1227, 1264)  # inclusive, now 0-based indexing

# Open and process the PDF
with pdfplumber.open(pdf_path) as pdf:
    for page_num in program_class_range:
        page = pdf.pages[page_num]
        page_text = page.extract_text()  # Extract text for the whole page

        # Split the text by lines
        lines = page_text.splitlines()
        
        # Accumulate course information
        course_text = ""  # Variable to accumulate text for each course entry
        current_course = {"course_code": "", "title": "", "credits": 0, "prerequisites": "", "description": "", "components": "", "notes": ""}
        
        for line in lines:
            # Check if a line is the start of a new course
            if re.match(r'^[A-Z]{4} \d{3}', line):
                # Process the accumulated course entry if it's not empty
                if course_text:
                    # Match the components of the course text using regex
                    course_match = course_pattern.search(course_text)
                    if course_match:
                        current_course["course_code"] = course_match.group(1)
                        current_course["title"] = course_match.group(2).strip()
                        current_course["credits"] = int(course_match.group(3))
                    
                    # Check for other components
                    prerequisite_match = prerequisite_pattern.search(course_text)
                    if prerequisite_match:
                        current_course["prerequisites"] = prerequisite_match.group(1).strip()
                    
                    description_match = description_pattern.search(course_text)
                    if description_match:
                        current_course["description"] = description_match.group(1).strip()
                    
                    component_match = component_pattern.search(course_text)
                    if component_match:
                        current_course["components"] = component_match.group(1).strip()
                    
                    notes_match = notes_pattern.search(course_text)
                    if notes_match:
                        current_course["notes"] = notes_match.group(1).strip()
                    
                    # Append to the list and reset
                    courses.append(current_course.copy())
                
                # Start accumulating for the new course
                course_text = line
            else:
                # Continue accumulating text for the current course entry
                course_text += "\n" + line
        
        # Handle the last course on the page
        if course_text:
            # Match the components of the course text using regex
            course_match = course_pattern.search(course_text)
            if course_match:
                current_course["course_code"] = course_match.group(1)
                current_course["title"] = course_match.group(2).strip()
                current_course["credits"] = int(course_match.group(3))
            
            # Check for other components
            prerequisite_match = prerequisite_pattern.search(course_text)
            if prerequisite_match:
                current_course["prerequisites"] = prerequisite_match.group(1).strip()
            
            description_match = description_pattern.search(course_text)
            if description_match:
                current_course["description"] = description_match.group(1).strip()
            
            component_match = component_pattern.search(course_text)
            if component_match:
                current_course["components"] = component_match.group(1).strip()
            
            notes_match = notes_pattern.search(course_text)
            if notes_match:
                current_course["notes"] = notes_match.group(1).strip()
            
            # Append the final course entry on the page
            courses.append(current_course.copy())

# Convert to DataFrame for easy viewing/manipulation
courses_df = pd.DataFrame(courses)
print(courses_df.head())


  course_code                             title  credits  \
0    NEUR 355      Fundamentals of Neuroscience        3   
1    NEUR 356  Research Methods in Neuroanatomy        3   
2    NEUR 416              Large Data Analytics        3   
3    NEUR 451      Learning and Neuroplasticity        3   
4    NEUR 458      Neuroscience Advanced Issues        3   

                                       prerequisites description components  \
0  One of the following courses must be completed...                          
1  The following courses must be completed previo...                          
2  The following courses must be completed previo...                          
3  The following courses must be completed previo...                          
4  The following courses must be completed previo...                          

  notes  
0        
1        
2        
3        
4        


In [109]:
course_match.group(0)

AttributeError: 'NoneType' object has no attribute 'group'

In [107]:
courses_df.prerequisites[0]

'One of the following courses must be completed previously:'

In [47]:
courses_df.iloc[0,:]

course_code                                               BIOL 200
title            or BIOL 201 or BIOL 202; or Cegep equivalent.\...
credits                                                          3
prerequisites                                                     
description                                                       
components                                                        
notes                                                             
Name: 0, dtype: object

In [48]:
import fitz  # PyMuPDF
import re
import pandas as pd

# Path to your PDF file
pdf_path = calendar_pdf_fname

# Regular expressions to match course structure
course_pattern = re.compile(r'([A-Z]{4} \d{3})\s+([^\n]+)')  # Matches course code and title
prerequisite_pattern = re.compile(r'Prerequisite[s]/Corequisite[s]:*:\s+([A-Z]{4} \d{3}(?:,? [A-Z]{4} \d{3})*)')

# List to store extracted courses
courses = []

In [49]:
def test_match(pdf_path,program_class_range=range(1228-1,1264)):
    with fitz.open(pdf_path) as pdf:
        for page_num in range(pdf.page_count):
            # print(page_num)
            if page_num in program_class_range:
                page = pdf[page_num]
                text = page.get_text()
                return text
                # for match in course_pattern.finditer(text):
                #     return match

# psych courses starts on 1229 (page of the pdf, not number at bottom of page)
program_class_range = range(1228-1,1264) #inclusive, now 0-based indexing

text = test_match(pdf_path,program_class_range)
for course in course_code_pattern.finditer(text):
    print(course)

<re.Match object; span=(1410, 1459), match='NEUR 355 Fundamentals of Neuroscience (3 credits)>
<re.Match object; span=(1460, 1513), match='NEUR 356 Research Methods in Neuroanatomy (3 cred>
<re.Match object; span=(1514, 1555), match='NEUR 416 Large Data Analytics (3 credits)'>


In [50]:
text

'Psychology Courses\nNeuroscience Courses\nPrerequisite/Corequisite:\nOne of the following courses must be completed previously: BIOL 200 or BIOL 201 or BIOL 202; or Cegep equivalent.\nDescription:\nThis course provides an introduction to the neural mechanisms that underlie behaviour. Topics include the structure\nand function of neurons, neural communication, an introduction to neuroanatomy and endocrinology, and the\nprocessing of sensory information. Students also learn how complex systems, such as the sensory and motor\nsystems, interact to produce behaviour.\nComponent(s):\nLecture\nNotes:\n• Students who have received credit for BIOL 383 or PSYC 355 or PSYC 358 may not take this course for credit.\nPrerequisite/Corequisite:\nThe following courses must be completed previously: NEUR 355; PSYC 315. Permission of the Department is required\nfor non-Neuroscience students.\nDescription:\nThe course examines research methods aimed at structural and functional brain systems. Analyses are

In [51]:
course_code_pattern = re.compile(r'([A-Z]{4} \d{3}) (.+?) \((\d+) credits\)')
prerequisite_pattern = re.compile(r'Prerequisite/Corequisite:\s*(.+)')
description_pattern = re.compile(r'Description:\s*(.+)', re.DOTALL)
component_pattern = re.compile(r'Component\(s\):\s*(.+)')
notes_pattern = re.compile(r'Notes:\s*(.+)', re.DOTALL)
course_code_pattern.search(text)

<re.Match object; span=(1410, 1459), match='NEUR 355 Fundamentals of Neuroscience (3 credits)>

In [52]:
import fitz  # PyMuPDF
import re
import pandas as pd


# Patterns to match specific information
course_code_pattern = re.compile(r'([A-Z]{4} \d{3}) (.+?) \((\d+) credits\)')
prerequisite_pattern = re.compile(r'Prerequisite/Corequisite:\s*(.+)')
description_pattern = re.compile(r'Description:\s*(.+)', re.DOTALL)
component_pattern = re.compile(r'Component\(s\):\s*(.+)')
notes_pattern = re.compile(r'Notes:\s*(.+)', re.DOTALL)

# List to store extracted course information
courses = []

# Open and process the PDF
with fitz.open(pdf_path) as pdf:
    text = ""
    for page_num in range(pdf.page_count):
        page = pdf[page_num]
        text += page.get_text()  # Get all text from the page

KeyboardInterrupt: 

In [ ]:
# Isolate the Neuroscience Courses section
start_index = text.find("Neuroscience Courses")
if start_index != -1:
    text = text[start_index:]

# Split each course based on patterns
course_sections = re.split(course_code_pattern, text)
for i in range(1, len(course_sections), 3):
    code = course_sections[i]
    title = course_sections[i + 1].strip()
    credits = int(course_sections[i + 2].strip().split()[0])  # Extract credit number

    # Initialize fields
    prerequisites = ""
    description = ""
    components = ""
    notes = ""

    # Extract prerequisites
    prereq_match = prerequisite_pattern.search(course_sections[i + 2])
    if prereq_match:
        prerequisites = prereq_match.group(1).strip()

    # Extract description
    description_match = description_pattern.search(course_sections[i + 2])
    if description_match:
        description = description_match.group(1).split("Component(s):")[0].strip()

    # Extract components
    component_match = component_pattern.search(course_sections[i + 2])
    if component_match:
        components = component_match.group(1).strip()

    # Extract notes
    notes_match = notes_pattern.search(course_sections[i + 2])
    if notes_match:
        notes = notes_match.group(1).strip()

    # Store course details in a dictionary
    courses.append({
        "course_code": code,
        "title": title,
        "credits": credits,
        "prerequisites": prerequisites,
        "description": description,
        "components": components,
        "notes": notes
    })

# Convert list of courses to DataFrame
courses_df = pd.DataFrame(courses)

# Display the DataFrame to verify extraction
print(courses_df.head())

# Optionally, save to CSV or JSON
courses_df.to_csv('neuroscience_courses.csv', index=False)
courses_df.to_json('neuroscience_courses.json', orient='reco

,course_code,title,description,prerequisites
0,NEUR 458,prior to taking this course.,,[]
1,NEUR 356,"or PSYC 310; PSYC 316, PSYC 351.",,[]
2,NEUR 356,or PSYC 310; PSYC 315. Permission of the,,[]
3,NEUR 451,Learning and Neuroplasticity (3 credits),,[]
4,NEUR 458,Neuroscience Advanced Issues (3 credits),,[]
...,...,...,...,...
201,PSYC 421,may not take this course for credit.,,[]
202,PSYC 422,may not take this course for credit.,,[]
203,PSYC 491,Honours Seminar Issues (3 credits),,[]
204,PSYC 495,Honours Thesis (6 credits),,[]


In [ ]:
# orig_df = courses_df.copy()
orig_df

,course_code,title,description,prerequisites
0,MATH 203,or MATH 209,,[]
1,MATH 200,and,,[]
2,BIOL 200,or BIOL 201 or BIOL 202,,[]
3,MATH 205,Note: Students may need one or more of MATH 20...,,[]
4,BIOL 201,or BIOL 202; three credits in,,[]
...,...,...,...,...
12397,SCEN 470,Advanced Elements of Production (6 credits),,[]
12398,SCEN 471,Advanced Elements of Production (3 credits),,[]
12399,DFTT 471,may not take this course for credit.,,[]
12400,SCEN 498,Special Topics in Scenography (3 credits),,[]


# NetworkX graph setup

In [ ]:
import networkx as nx

# Initialize directed graph
course_graph = nx.DiGraph()

# Add courses as nodes with attributes
course_graph.add_node("PSYC 200", title="Intro to Psychology", credits=3, level="200")
course_graph.add_node("PSYC 300", title="Advanced Psychology", credits=3, level="300")

# Add edges to represent prerequisites
course_graph.add_edge("PSYC 200", "PSYC 300")  # PSYC 200 is a prerequisite for PSYC 300
